In [1]:
# To export as GUI:
# Use Anaconda Prompt and cd to file location
# Run pyinstaller.exe FILENAME.py --noconsole

import QuantLib as ql
import tkinter as tk
from tkinter import ttk
# Following two lines fix blurry tkinter text
from ctypes import windll
windll.shcore.SetProcessDpiAwareness(1)

0

### Black-Scholes Digital Option Pricer GUI

In [2]:
root = tk.Tk()
root.title("Black-Scholes Digital Option Pricer")
root.configure(bg="white")
root.resizable(False, False)
root.geometry("840x430")

style = ttk.Style()
radbtn_style = ttk.Style()
cb_style = ttk.Style()
style.configure("BW.TLabel", foreground="black", background="white")
radbtn_style.configure("Wild.TRadiobutton", background="white")
cb_style.configure("Wild.TCheckbutton", background="white")
    
def radbtn_selection():
    
    try:
        strike = float(strike_entry.get())
        ex_date_str = ex_entry.get()
    
        # Change date from str format to ql.Date format    
        ex_date = ql.DateParser.parseFormatted(ex_date_str, "%Y-%m-%d") 
        
        payoff_val = float(payoff_entry.get())
        
        # Put radiobutton selected, price a put
        if rb_state.get() == "Put":
            binary_option = ql.VanillaOption(ql.CashOrNothingPayoff(ql.Option.Put, strike, payoff_val), 
                                             ql.EuropeanExercise(ex_date))
        # Call radiobutton selected, price a call
        elif rb_state.get() == "Call":
            binary_option = ql.VanillaOption(ql.CashOrNothingPayoff(ql.Option.Call, strike, payoff_val), 
                                             ql.EuropeanExercise(ex_date))
    
        return binary_option
    
    except (ValueError, RuntimeError):
        pass
    

def calc_npv_greeks():
    
    try:
        # Retrieve data from corresponding entry boxes
        vol = vol_entry.get()
        rfr = rfr_entry.get()
        if "%" in vol:
            vol = float(vol.strip("%")) / 100
        if "%" in rfr:
            rfr = float(rfr.strip("%")) / 100
        else:    
            vol = float(vol)
            rfr = float(rfr)
        spot = float(spot_entry.get())

        eval_date_str = eval_entry.get()
        eval_date = ql.DateParser.parseFormatted(eval_date_str, "%Y-%m-%d")     

        ql.Settings.instance().evaluationDate = eval_date
        #ql.Settings.instance().setEvaluationDate(eval_date)

        option = radbtn_selection()

        # Volatility
        sigma = ql.SimpleQuote(vol)
        # Risk-free rate
        r = ql.SimpleQuote(rfr)
        # Underlying asset price (spot)
        u = ql.SimpleQuote(spot)
        
        # Calendar - US, day count convention - actual/360
        risk_free_curve = ql.FlatForward(0, ql.UnitedStates(), ql.QuoteHandle(r), ql.Actual360())
        volatility = ql.BlackConstantVol(0, ql.UnitedStates(), ql.QuoteHandle(sigma), ql.Actual360())

        process = ql.BlackScholesProcess(ql.QuoteHandle(u), 
                                         ql.YieldTermStructureHandle(risk_free_curve),
                                         ql.BlackVolTermStructureHandle(volatility))

        engine = ql.AnalyticEuropeanEngine(process)
        option.setPricingEngine(engine)
        
        dp = 5
        npv_value.set(round(option.NPV(), dp))
        delta_value.set(round(option.delta(), dp))
        gamma_value.set(round(option.gamma(), dp))
        vega_value.set(round(option.vega(), dp))
        theta_value.set(round(option.theta(), dp))
        
    except (ValueError, AttributeError, UnboundLocalError, RuntimeError):
        pass


# Widgets ---------------------------------------------
vol = ttk.Label(text="Volatility", style="BW.TLabel")
rfr = ttk.Label(text="Risk-free Rate", style="BW.TLabel")
spot = ttk.Label(text="Asset Spot Price", style="BW.TLabel")

vol_entry = ttk.Entry()
rfr_entry = ttk.Entry()
spot_entry = ttk.Entry()

eval_date = ttk.Label(text="Evaluation Date (yyyy-mm-dd)", style="BW.TLabel")
ex_date = ttk.Label(text="Exercise Date (yyyy-mm-dd)", style="BW.TLabel")
strike_p = ttk.Label(text="Option Strike Price", style="BW.TLabel")
payoff = ttk.Label(text="Option Payoff", style="BW.TLabel")

eval_entry = ttk.Entry()
ex_entry = ttk.Entry()
strike_entry = ttk.Entry()
payoff_entry = ttk.Entry()

npv = ttk.Label(text="Option NPV:", style="BW.TLabel")
npv_value = tk.StringVar()
npv_label = ttk.Label(textvariable=npv_value)
calc_button = ttk.Button(text="Option NPV & Greeks", command=calc_npv_greeks)

delta_value = tk.StringVar()
delta_label = ttk.Label(textvariable=delta_value)
gamma_value = tk.StringVar()
gamma_label = ttk.Label(textvariable=gamma_value)
vega_value = tk.StringVar()
vega_label = ttk.Label(textvariable=vega_value)
theta_value = tk.StringVar()
theta_label = ttk.Label(textvariable=theta_value)

rb_state = tk.StringVar()
call_btn = ttk.Radiobutton(text="Call Option", style="Wild.TRadiobutton", value="Call",
                           variable=rb_state, command=radbtn_selection)
put_btn = ttk.Radiobutton(text="Put Option", style="Wild.TRadiobutton", value="Put", 
                          variable=rb_state, command=radbtn_selection)

# Grid packing ------------------------------------
# Padding parameters
all_padding = {"padx":(50,5), "pady":0}
y_padding = {"padx":(50,5), "pady":(20,0)}

# Zeroth row
ttk.Label(text="Black-Scholes Digital Option Pricer", font=("Calibri Light", 11, "bold"), style="BW.TLabel").grid(row=0, columnspan=3, column=0, **{"padx":(55,0), "pady":(20,0)})

# First row
vol.grid(row=1, column=0, **y_padding)
rfr.grid(row=1, column=1, **y_padding)
spot.grid(row=1, column=2, **y_padding)

# Second row
vol_entry.grid(row=2, column=0, **all_padding)
rfr_entry.grid(row=2, column=1, **all_padding)
spot_entry.grid(row=2, column=2, **all_padding)

# Third row
eval_date.grid(row=3, column=0, **y_padding)
ex_date.grid(row=3, column=1, **y_padding)
strike_p.grid(row=3, column=2, **y_padding)

# Fourth row

eval_entry.grid(row=4, column=0, **all_padding)
ex_entry.grid(row=4, column=1, **all_padding)
strike_entry.grid(row=4, column=2, **all_padding)

# Fifth row
ttk.Label(text="Delta:", style="BW.TLabel").grid(row=5, column=0, **y_padding)
ttk.Label(text="Gamma:", style="BW.TLabel").grid(row=5, column=1, **y_padding)
payoff.grid(row=5, column=2, **y_padding)

# Sixth row
delta_label.grid(row=6, column=0, **all_padding)
delta_label.config(background="white")
gamma_label.grid(row=6, column=1, **all_padding)
gamma_label.config(background="white")
payoff_entry.grid(row=6, column=2, **all_padding)

# Seventh row
ttk.Label(text="Vega:", style="BW.TLabel").grid(row=7, column=0, **y_padding)
ttk.Label(text="Theta:", style="BW.TLabel").grid(row=7, column=1, **y_padding)
call_btn.grid(row=7, column=2, **{"padx":(51.5,5), "pady":(20,0)})

# Eigth row
vega_label.grid(row=8, column=0, **all_padding)
vega_label.config(background="white")
theta_label.grid(row=8, column=1, **all_padding)
theta_label.config(background="white")
put_btn.grid(row=8, column=2, **all_padding)

# Ninth row
npv.grid(row=9, column=0, **y_padding)
calc_button.grid(row=9, rowspan=2, column=1, **y_padding)

# Tenth row
npv_label.grid(row=10, column=0, **all_padding)
npv_label.config(background="white")
ttk.Label(text="By Andy Leo", font=("Calibri Light", 11, "italic"), style="BW.TLabel").grid(row=10, column=2, **{"padx":(200,0), "pady":(0,0)})

root.mainloop()